# Richardson–Lucy deconvolution algorithm

Restoration of digital images from their degraded measurement has always been a problem of
great interest. 

A specific solution to the problem of image restoration is generally determined
by the nature of degradation phenomena. So it is highly dependent on the nature of the
noise present there.


Given the noise function, one can use the Richardson-Lucy Algorithm
to restore the degraded image. This algorithm was introduced by W.H. Richardson (1972)
and L.B. Lucy (1974).

The Point Spread Function describes the response of an imaging system to a point source or point object

Richardson Lucy is probably the most commonly used deconvolution algorithm, it works well in most cases, especially if noise is relatively low, and you are able to create a reasonably accurate PSF. The standard Richardson Lucy algorithm is not “blind”, so you need to create the PSF(Point Spread Function) using a theoretical model, or by measuring it.

In [ ]:
import imageio
import DeconUtility
import OpenCLDeconvUtility
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import time
import YacuDecuUtility
import itk

# open image and psf
imgName='/content/img1.jpg'
#psfName='/home/bnorthan/code/images/PSF-Bars-stack-cropped.tif'

img=io.imread(imgName)
psf=psf = np.ones((5, 5)) / 25

extDims=DeconUtility.nextPow2(img.shape)

(img, padding)=DeconUtility.padNDImage(img, extDims, 'reflect')
(psf, padding)=DeconUtility.padNDImage(psf, extDims, 'constant')

print(img.shape)
print(psf.shape)

#matplotlib inline
fig, ax = plt.subplots(1,2)
ax[0].imshow(img.max(axis=0))
ax[0].set_title('img')

ax[1].imshow(psf.max(axis=0))
ax[1].set_title('psf')

img=img.astype(np.float32)
psf=psf.astype(np.float32)
psf=psf/psf.sum();
shifted_psf = np.fft.ifftshift(psf)
result = np.copy(img);
normal=np.ones(img.shape).astype(np.float32)

lib=YacuDecuUtility.getYacuDecu()
print('GPU Memory is',lib.getTotalMem())

start=time.time()
lib.deconv_device(100, int(img.shape[0]), int(img.shape[1]), int(img.shape[2]), img, shifted_psf, result, normal)
end=time.time()
cudatime=end-start
print('time is',end-start)

libcl=OpenCLDeconvUtility.getArrayFire()
deconcl=img.copy()

start=time.time()
libcl.deconv(100,img.shape[2], img.shape[1], img.shape[0], img, shifted_psf, deconcl, img);
end=time.time()
cvtime=end-start
print('cl time', end-start)

#matplotlib inline
fig, ax = plt.subplots(1,3)
ax[0].imshow(img.max(axis=0))
ax[0].set_title('img')

ax[1].imshow(result.max(axis=0))
ax[1].set_title('result')

ax[2].imshow(deconcl.max(axis=0))
ax[2].set_title('opencl result')